In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
states_BR = ['AL',
 'BA',
 'CE',
 'MA',
 'PB',
 'PE',
 'PI',
 'SE',
 'RN',
 'SP',
 'MG',
 'RJ',
 'ES',
 'AM',
 'AP',
 'TO',
 'RR',
 'RO',
 'AC',
 'PA',
 'DF',
 'GO',
 'MT',
 'MS',
 'RS',
 'SC',
 'PR']

Serão atualizadas apenas as últimas 10 semanas dos dados da coluna de casos prováveis. Os dados atualizados serão uma proporção dos casos notificados estimados pelo nowcast. Essa proporção é calculada a partir da série dos estados usando os dados de 2024 (excluindo as últimas 10 semanas que podem estar subnotificadas)

In [3]:
df_prop = pd.read_csv('casos_updated_se_38.csv.gz', index_col = 'Unnamed: 0')

df_prop.data_iniSE = pd.to_datetime(df_prop.data_iniSE)

df_prop.set_index('data_iniSE', inplace = True)

df_prop = df_prop.groupby('uf').resample('W-SUN').sum().drop(['regional', 'uf'], axis =1).reset_index()

df_prop.head()

,uf,data_iniSE,casos_est,casprov
0,AC,2023-06-04,206.0,60.0
1,AC,2023-06-11,153.0,27.0
2,AC,2023-06-18,176.0,41.0
3,AC,2023-06-25,146.0,29.0
4,AC,2023-07-02,133.0,27.0


In [4]:
end_date_prop = df_prop.data_iniSE.max() - pd.DateOffset(weeks=10)
begin_date_prop = df_prop.data_iniSE.max() - pd.DateOffset(weeks=34)

In [5]:
print(begin_date_prop)
print(end_date_prop)

2024-01-21 00:00:00
2024-07-07 00:00:00


In [6]:
prop_state = dict()

for state in states_BR: 
    df_filter = df_prop.loc[(df_prop.uf == state) & (df_prop.data_iniSE >= begin_date_prop) & (df_prop.data_iniSE <= end_date_prop)]


    prop_state[state]= np.mean(df_filter['casprov']/df_filter['casos_est'])

In [7]:
#pd.DataFrame(prop_state).T.rename(columns = {0: 'prop'})

In [8]:
df_atual = pd.read_csv('dengue_up.csv.gz', index_col = 'date', usecols = ['date', 'regional_geocode', 'casos', 'uf'])

df_atual.index = pd.to_datetime(df_atual.index)

df_atual = df_atual.groupby('regional_geocode').resample('W-SUN').sum().drop('regional_geocode', axis = 1).reset_index()

df_atual['uf'] = df_atual['uf'].str[:2]

#df_atual = df_atual.loc[df_atual.date <= end_date_prop]
df_atual = df_atual.loc[df_atual.date < '2024-01-01']

df_atual.head()

,regional_geocode,date,casos,uf
0,11001,2010-01-03,367,RO
1,11001,2010-01-10,290,RO
2,11001,2010-01-17,283,RO
3,11001,2010-01-24,217,RO
4,11001,2010-01-31,221,RO


Atualizar os dados: 

In [9]:
df_up = pd.read_csv('casos_updated_se_38.csv.gz', index_col = 'Unnamed: 0')

df_up.data_iniSE = pd.to_datetime(df_up.data_iniSE)

df_up.set_index('data_iniSE', inplace = True)

df_up_no_delay =  df_up.loc[(df_up.index <= end_date_prop) & (df_up.index >= '2024-01-01')].rename(columns = {'casprov': 'casos'}).drop('casos_est',axis =1)

df_up_no_delay = df_up_no_delay.reset_index().rename(columns= {'regional': 'regional_geocode',
                                                              'data_iniSE':'date'})[['regional_geocode',
                                                                                               'date', 'casos', 'uf']]
df_up_cor = df_up.loc[df_up.index > end_date_prop].reset_index().rename(columns= {'regional': 'regional_geocode',
                                                              'data_iniSE':'date'})

#df_up = df_prop.groupby('uf').resample('W-SUN').sum().drop(['regional', 'uf'], axis =1).reset_index()

df_up_cor.head()

,date,regional_geocode,casos_est,casprov,uf
0,2024-07-14,27001,348.5,294.0,AL
1,2024-07-21,27001,259.0,193.0,AL
2,2024-07-28,27001,198.0,144.0,AL
3,2024-08-04,27001,208.5,140.0,AL
4,2024-08-11,27001,156.0,81.0,AL


In [10]:
def up_data(df_up, prop_state, state): 

    df_up_ = df_up.loc[df_up.uf == state]

    df_up_.loc[:, 'casos'] = prop_state[state]*df_up_['casos_est']
    
    #df_up_.loc[:, 'casos'] = df_up_[['casos_', 'casprov']].max(axis=1)
    
    #df_up_ = df_up_.drop(['casos_est', 'casos_', 'casprov'],axis =1)
    df_up_ = df_up_.drop(['casos_est', 'casprov'],axis =1)
    
    df_up_ = df_up_[['regional_geocode', 'date','casos', 'uf']]

    return df_up_

In [11]:
#df_update = df_atual
df_update = pd.concat([df_atual, df_up_no_delay])
for state in states_BR:

    df_update  = pd.concat([df_update, up_data(df_up_cor, prop_state, state)])


df_update.head()

/var/folders/ch/kxpr39wx44v97968yr_4hmch0000gn/T/ipykernel_24903/1965884426.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_up_.loc[:, 'casos'] = prop_state[state]*df_up_['casos_est']
/var/folders/ch/kxpr39wx44v97968yr_4hmch0000gn/T/ipykernel_24903/1965884426.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_up_.loc[:, 'casos'] = prop_state[state]*df_up_['casos_est']
/var/folders/ch/kxpr39wx44v97968yr_4hmch0000gn/T/ipykernel_24903/1965884426.py:5: SettingWithCopyWarning: 
A value is trying to be

,regional_geocode,date,casos,uf
0,11001,2010-01-03,367.0,RO
1,11001,2010-01-10,290.0,RO
2,11001,2010-01-17,283.0,RO
3,11001,2010-01-24,217.0,RO
4,11001,2010-01-31,221.0,RO


In [12]:
df_update.date.min()

Timestamp('2010-01-03 00:00:00')

In [13]:
df_update.date.max()

Timestamp('2024-09-15 00:00:00')

In [14]:
df_update.to_csv('dengue_update2.csv.gz')